In [10]:
import datasets
from datasets import load_from_disk

In [11]:
ds = load_from_disk("test.hf/")
print(ds)

Dataset({
    features: ['prompt', 'completion', 'inputs', 'output'],
    num_rows: 1
})


In [12]:
sample = next(iter(ds))

In [6]:
from datasets import load_dataset
import os
from datasets import Dataset
import json
import asyncio
import aiohttp
import multiprocessing
import time
import io
import sys
import builtins
import re
import random
from tqdm import tqdm

In [7]:
def run_code(code: str, test_in: str, output_queue):
    start = time.time()
    compiled = re.findall("```python\n(.*?)\n```", code, re.DOTALL)
    if len(compiled) > 0:
        code = compiled[-1]
    else:
        return "CODE NOT FOUND"
    if isinstance(test_in, list):
        test_in = " ".join(str(s) for s in test_in)

    # Mock input
    input_lines = iter(test_in.splitlines())
    builtins.input = lambda: next(input_lines)

    # Capture output
    buf = io.StringIO()
    sys.stdout = buf
    try:
        exec(code)
    except Exception as _:
        output_queue.put(("RUNTIME ERROR\n", 0))
        return
    finally:
        sys.stdout = sys.__stdout__

    output_queue.put(
        (
            {
                "output": buf.getvalue(),
                "time": time.time() - start,
            },
            0,
        )
    )

In [ ]:
def test_code(
    code: str, inputs: list[str], outputs: list[str], time_limit=5000
) -> bool:
    for i in range(len(inputs)):
        output_queue = multiprocessing.Queue()
        p = multiprocessing.Process(target=run_code, args=(code, inputs, output_queue))
        p.start()
        p.join(timeout=time_limit)

        if p.is_alive():
            p.terminate()
            print("infinite loop")
            return False
        else:
            try:
                out, _ = output_queue.get(timeout=1)
            except:
                print("failed to get output")
                return False

        if not(isinstance(out, dict) or out["output"].rstrip() == outputs[i].rstrip()):
            print("wrong output")
            return False

    print("passed")
    return True

In [9]:
model_code ="""import math
t = int(input())
for _ in range(t):
    n = int(input())
    if n == 0:
        print(-1)
        continue
    res = [0] * n
    next_i = n - 1
    while next_i >= 0:
        s = math.isqrt(next_i)
        if s * s < next_i:
            s += 1
        m = s * s
        start = max(0, m - next_i)
        end = next_i
        for i in range(start, end + 1):
            res[i] = m - i
        next_i = start - 1
    # Check if all elements are unique and in the correct range
    valid = True
    seen = [False] * n
    for num in res:
        if num < 0 or num >= n or seen[num]:
            valid = False
            break
        seen[num] = True
    if valid:
        print(' '.join(map(str, res)))
    else:
        print(-1)"""

print(test_code(model_code, ["3\n3\n4\n7\n"], ["1 0 2 \n0 3 2 1 \n1 0 2 6 5 4 3 \n"]))

Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=75, pipe_handle=82)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/CompetitiveLLM/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/anaconda3/envs/CompetitiveLLM/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run_code' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


False
